# Rede Neural

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# DEFINI VARIÁVEIS

In [2]:
def inicia_variaveis():
    
    global tempoInicialGeral, tempoFinalGeral, identificadas, baixo, alto, fator_imgs, altura_minima_ponto, altura_maxima_ponto
    global corretas, erradas, info, sensibilidade, mediaHistograma, LER_IMG, ESCREVE_IMG, distancia_pontos_raio_px, delta_distancia_lidar, reducao_caixa_fruta
    global TIPO_BASE, limiarLarguraPedunculo, limiarAlturaPedunculo, NOME_IMAGEM, NOME_JSON, NOME_TXT, METODO, min_samples,metodo_pontos_curva,altura_caixa_pedunculo, EXECUTA_VALIDACAO, QUANTIDADE_IMAGEM,POSICAO_PONTO
    
    tempoInicialGeral = 0
    tempoFinalGeral = 0
    
    corretas =        []
    erradas =         []
    info =            []
    sensibilidade =   []
    mediaHistograma = []

    limiarAlturaPedunculo  = 0.3
    limiarLarguraPedunculo = 0.2

    baixo = [2, 0, 0]
    alto  = [31, 255, 255]

    distancia_pontos_raio_px = 2

    fator_imgs = 7.5
    
    altura_minima_ponto = 3
    altura_maxima_ponto = 4
    
    delta_distancia_lidar = 10
    
    reducao_caixa_fruta = 0.2
    
    altura_caixa_pedunculo = 10
    
    EXECUTA_VALIDACAO = True
    
    min_samples = 4
    metodo_pontos_curva = "IsolationForest"
    #metodo_pontos_curva = "Outliners"
    
    TIPO_BASE = "3D"
    
    POSICAO_PONTO = "top"
    
    #METODO = "DBSCAN"
    #METODO = "Kmeans"
    METODO = "Heuristica"
    
    ##################2222222222222222222222222222222222DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD############################
    
    #PASTA = "VALID_CUT_POINT_2D/"
    
    #LER_IMG           = "VI2D_MANGO_LEAFLESS_SEM_FOLHA/"
    #NOME_IMAGEM       = "_MANGO_LEAFLESS"
    #QUANTIDADE_IMAGEM = 242
    #ESCREVE_IMG = "RESULTADOS_IMAGENS_2D/VI2D_MANGO_LEAFLESS_SEM_FOLHA/VI2D_MANGO_LEAFLESS_" + METODO + "/"
    
    #LER_IMG          = "VI2D_MANGO_LEAFY_BACKGROUND_COM_FOLHA/"
    #NOME_IMAGEM      = "_MANGO_LEAFY_BACKGROUND"
    #QUANTIDADE_IMAGEM = 201
    #ESCREVE_IMG = "RESULTADOS_IMAGENS_2D/VI2D_MANGO_LEAFY_BACKGROUND_COM_FOLHA/VI2D_MANGO_LEAFY_BACKGROUND_" + METODO + "/"
    
    #LER_IMG          = "VI2D_MANGO_OCCLUDED_MANGA_OCLUSA/"
    #NOME_IMAGEM      = "_MANGO_OCCLUDED"
    #QUANTIDADE_IMAGEM = 145
    #ESCREVE_IMG = "RESULTADOS_IMAGENS_2D/VI2D_MANGO_OCCLUDED_MANGA_OCLUSA/VI2D_MANGO_OCCLUDED_" + METODO + "/"
    
    #LER_IMG          = "VI2D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/"
    #NOME_IMAGEM      = "_STEM_OCCLUDED"
    #QUANTIDADE_IMAGEM = 166
    #ESCREVE_IMG = "RESULTADOS_IMAGENS_2D/VI2D_STEM_OCCLUDED_PEDUNCLO_OCLUSO/VI2D_STEM_OCCLUDED_" + METODO + "/"
    
    
    ##################3333333333333333333333333333333DDDDDDDDDDDDDDDDDDDDDDDDDDDDDD#################################
    
    PASTA = "VALID_CUIT_POINT_3D/"
    
    LER_IMG = "VI3D_MANGO_LEAFLESS_SEM_FOLHA/"
    NOME_IMAGEM = "_MANGO_LEAFLESS"
    ESCREVE_IMG = "VALID_CUIT_POINT_3D/RESULTADO/"

    
    NOME_JSON   = "_MANGO_MAP_JSON"
    LER_IMG = PASTA + LER_IMG    
    NOME_TXT = ESCREVE_IMG[:-1] + "_RESULTADOS_" + NOME_IMAGEM
    NOME_TXT = NOME_TXT.upper()

# FUNÇÃO PRINCIPAL

In [6]:
import cv2 
import numpy as np
import time
import traceback
import math
import json
import os

import PC_SVCF as pc
import Lidar as lidar
import Valida_SVCF as vl

id_imagem = 1
QUANTIDADE_IMAGEM = 2

inicia_variaveis()

print("Executando...")
print("Log..." + NOME_TXT)
arquivo = open(NOME_TXT + ".csv", "w")

identificadas = 0
naoIdentificadas = 0

pontoCorreto = 0
pontoErrado = 0
quantidadeManga = 0

if(TIPO_BASE == "2D"):
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,HUE,%_MANGA,%_PONTO,MEDIA\n")
    
    tamanhoLetraTexto = 0.6
    fatorLetra = 20
    
else:
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,FATOR_CM_PX,DISTÂNCIA_MANGA,HUE,%_MANGA,%_PEDÚNCULO,DISTÂNCIA_PONTO,DIFERENÇA_HORIZONTAL_P,DIFERENÇA_VERTICAL_P,DIFERENÇA_VERTICAL_C,%_PONTO,MÉDIA\n")
    
    tamanhoLetraTexto = 1.0
    fatorLetra = 35

tempoInicialGeral = time.time()

pontosJson = ""
fator_cm_px = ""

media_final = [ ]

while(id_imagem <= 153):
    
    inicia_variaveis()

    print("############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################\n")
    
    caminho_imagem = LER_IMG + str(id_imagem) + NOME_IMAGEM +  ".jpg"
    print(f"Caminho da imagem: {caminho_imagem}")
    
    imagem_original = cv2.imread(LER_IMG + str(id_imagem) + NOME_IMAGEM + ".jpg")
        
    caminho_json = LER_IMG + str(id_imagem) + NOME_JSON + ".json"
    print(f"Caminho do json:   {caminho_json}\n")
    
    if(TIPO_BASE == "3D"):

        pontosJson = lidar.organizaJSON(caminho_json)
        
    #Copia a imagem original
    imagem_original_copia = np.array(imagem_original)
    
    imagem_detectada = imagem_original_copia.copy() #USADA DURANTE A EXECUÇÃO DO CÓDIGO
    imagem_pedunculo = imagem_original_copia.copy() #USADA PARA A ÁREA DO PEDÚNCULO
    
    #Copia a imagem na versão reduzida
    imagem_original_reduzida_aux = cv2.resize(imagem_original, (192, 256))
    imagem_original_reduzida = np.array(imagem_original_reduzida_aux.astype(np.uint8))
    
    ################################ CONFIGURAÇÕES DA REDE NEURAL ################################
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(imagem_original_copia, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    
    detection_threshold = 0.8

    viz_utils.visualize_boxes_and_labels_on_image_array(
                imagem_detectada,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                skip_scores= True,
                skip_labels= True,
                skip_boxes= True,
                agnostic_mode=False)
  
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
        
    width =  imagem_detectada.shape[1]
    height = imagem_detectada.shape[0]
        
    sensibilidade.append(scores)
    
    quantidade_fruta_identificadas = len(scores)
    
    ################################ INICIO DO PROCESSO DE LOCALIZAÇÃO DO PONTO DE CORTE ################################
    
    if(quantidade_fruta_identificadas > 0):
                
        print(f"Imagem: {id_imagem} - Quantidade de fruta identificada: {quantidade_fruta_identificadas}\n")
                
        try:
            
            identificadas += 1
            
            posicaoTextoY = fatorLetra
            
            for id_manga_localizada, box in enumerate(boxes):
                
                arquivo.write(f"{str(id_imagem)},")
                
                quantidadeManga += 1
                                
                print(f"Manga localizada: {id_manga_localizada}")
                
                if(id_manga_localizada < 2):
                    
                    arquivo.write(f"{str(id_manga_localizada)},")
                    arquivo.write(f"{str(round(scores[id_manga_localizada], 4))},")
                                                            
                    ################################# ÁREA DA MANGA #################################

                    roi = box * [height, width, height, width]
                    
                    xt_ia_manga = roi[1]
                    yt_ia_manga = roi[0]
                    
                    xb_ia_manga = roi[3]
                    yb_ia_manga = roi[2]

                    xt_ia_manga = int(xt_ia_manga)
                    xb_ia_manga = int(xb_ia_manga)
                    yt_ia_manga = int(yt_ia_manga)
                    yb_ia_manga = int(yb_ia_manga)
                    
                    cv2.putText(imagem_detectada, str(id_manga_localizada), (xt_ia_manga, yt_ia_manga - 5), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)
                                        
                    centroCaixaFrutaX = int((xt_ia_manga + xb_ia_manga) / 2)
                    centroCaixaFrutaY = int((yt_ia_manga + yb_ia_manga) / 2)
                    
                    centroCaixaFrutaXLidar = int(centroCaixaFrutaX / fator_imgs)
                    centroCaixaFrutaYLidar = int(centroCaixaFrutaY / fator_imgs)
                    
                    media_imagem = [ ]
                    
                    if(TIPO_BASE == "3D"):
                    
                        #Função para encontrar o fator cm_px referente a manga
                        fator_cm_px = pc.obter_cm_px(4, 2, centroCaixaFrutaXLidar, centroCaixaFrutaYLidar, pontosJson)
                                                
                        arquivo.write(f"{str(round(fator_cm_px, 4))},")

                        print(f"Fator cm_px: {fator_cm_px:.5f}")
                                            
                        altura_caixa_pedunculo = 2.0 / fator_cm_px * fator_imgs
                        
                        print(f"Altura caixa: {int(altura_caixa_pedunculo)}")

                        distanciaFruta = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, centroCaixaFrutaYLidar), pontosJson)

                        print(f"Distancia Manga: {distanciaFruta:.2f} cm")
                        
                        arquivo.write(f"{str(round(distanciaFruta, 2))},")
                        
                        cv2.putText(imagem_detectada, "Distancia Manga - " + str(id_manga_localizada) + ": "+ str(round(distanciaFruta, 2)) + " cm", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                        posicaoTextoY += fatorLetra

                        
                    #Faz um corte na área da manga
                    areaManga = imagem_detectada[yt_ia_manga:yb_ia_manga, xt_ia_manga:xb_ia_manga]
                    
                    #Circulo no centro da área da manga
                    cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(centroCaixaFrutaY)), 5, (255, 0, 90), -1)
                    
                    #Desenha um retangulo na área da manga
                    cv2.rectangle(imagem_detectada, (xt_ia_manga, yt_ia_manga), (xb_ia_manga, yb_ia_manga), (255,0,0), 2)
                    
                    
                    ################################# ÁREA DO PEDÚNCULO ################################# 

                    #Chama a função que prevê a área do pedúnculo
                    areaPedunculo = pc.preveAreaPedunculo(xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga, limiarLarguraPedunculo, limiarAlturaPedunculo, -altura_caixa_pedunculo)
                    
                    #x1                y1                 x2                 y2
                    xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

                    #Desenha um retangulo na área do pedúnculo        
                    cv2.rectangle(imagem_detectada, (xt_area_pedunculo, yt_area_pedunculo), (xb_area_pedunculo, yb_area_pedunculo), (0, 0, 255), 2)

                    #Coordenadas da área da manga
                    TopLeftX = xt_area_pedunculo
                    TopLeftY = yb_area_pedunculo
                    
                    #Faz o corte na área prevista do pedúnculo
                    corteAreaPedunculo = imagem_pedunculo[yb_area_pedunculo:yt_area_pedunculo, xt_area_pedunculo:xb_area_pedunculo]                    
                    
                    if(len(corteAreaPedunculo) < 1):
                        
                        print("Área do pedúnculo muito pequena")
                        
                    else:
                    
                        #Circulo no centro do topo da área da fruta
                        cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(yt_ia_manga)), 5, (0, 0, 255), -1)

                        if(TIPO_BASE == "3D"):

                            #Distancia do centro do topo da caixa do fruta
                            distanciaTopoCaixaFruto = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, int(yt_ia_manga / fator_imgs)), pontosJson)

                            print(f"Distancia Topo Caixa: {distanciaTopoCaixaFruto:.2f} cm")

                        #Função que encontra o ponto de corte final de acordo com os parâmetros
                        ponto_final = pc.localiza_ponto_final(str(id_imagem),
                                                str(id_manga_localizada),
                                                corteAreaPedunculo, 
                                                baixo, 
                                                alto, 
                                                METODO, 
                                                ESCREVE_IMG, 
                                                True, 
                                                POSICAO_PONTO, 
                                                7, 
                                                min_samples, 
                                                TIPO_BASE, 
                                                metodo_pontos_curva, 
                                                TopLeftX, 
                                                TopLeftY, 
                                                pontosJson, 
                                                altura_minima_ponto, 
                                                (centroCaixaFrutaX, centroCaixaFrutaY), 
                                                (centroCaixaFrutaX, yt_ia_manga))

                        pontoX, pontoY, valorHue, distanciaPonto = ponto_final[0], ponto_final[1], ponto_final[2], ponto_final[3]

                        arquivo.write(f"{str(valorHue)},")

                        cv2.circle(imagem_detectada, (pontoX, pontoY), 6, (255, 0, 255), -2)
                        cv2.line(imagem_detectada, (pontoX, pontoY), (centroCaixaFrutaX, yt_ia_manga), (255, 0, 255), 2)
                        cv2.circle(imagem_detectada, (pontoX, pontoY), 8, (255, 0, 0), 2)
                        
                        if(EXECUTA_VALIDACAO):
                            
                            #Função para validar o sistema
                            validacao = vl.valida_svcf(imagem_detectada, 
                                           None,
                                           id_imagem,
                                           id_manga_localizada,
                                           TIPO_BASE,
                                           (xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga),
                                           (xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo),
                                           caminho_json, 
                                           fator_cm_px,
                                           (pontoX, pontoY),
                                           altura_minima_ponto,
                                           distanciaPonto, 
                                           0, 
                                           3)

                            por_manga, por_pedunculo, por_ponto, distancia_hor, distancia_vert, distanciaPontoConhecido = validacao[0], validacao[1], validacao[2], validacao[3], validacao[4], validacao[5]

                            arquivo.write(f"{str(por_manga)},")

                            print(f"Porcentagem Manga:     {por_manga} %")
                            cv2.putText(imagem_detectada, "Precisao Manga: "+ str(por_manga) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra

                            if(por_pedunculo != -1):

                                print(f"Porcentagem Pedunculo: {por_pedunculo} %")
                                print(f"Distancia Horizontal:  {distancia_hor} cm")
                                print(f"Distancia Vertical:    {distancia_vert} cm")

                                arquivo.write(f"{str(por_pedunculo)},")
                                arquivo.write(f"{str(round(distanciaPonto, 2))},")
                                arquivo.write(f"{str(distancia_hor)},")
                                arquivo.write(f"{str(distancia_vert)},")

                                cv2.putText(imagem_detectada,"Precisao Pedunculo: " + str(por_pedunculo) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                posicaoTextoY += fatorLetra

                                cv2.putText(imagem_detectada, "Distancia Hor: " + str(distancia_hor) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                posicaoTextoY += fatorLetra
                                
                                cv2.putText(imagem_detectada, "Distancia Ver: " + str(distancia_vert) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                posicaoTextoY += fatorLetra

                            print(f"Porcentagem Ponto:     {por_ponto} %")
                            cv2.putText(imagem_detectada, "Precisao Ponto: " + str(por_ponto) + " %", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra
                            
                            if(por_ponto == 1):
                                
                                pontoCorreto += 1
                                
                            else:
                                
                                pontoErrado += 1

                            if(TIPO_BASE == "3D"):

                                cv2.putText(imagem_detectada, "Altura Ponto IA: " + str(distanciaPonto) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                posicaoTextoY += fatorLetra
                                
                                cv2.putText(imagem_detectada, "Altura Ponto Label: " + str(distanciaPontoConhecido) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                                posicaoTextoY += fatorLetra
                                
                                arquivo.write(f"{str(round(abs(distanciaPontoConhecido - distanciaPonto), 2))},")

                            arquivo.write(f"{str(por_ponto)},")

                            if(TIPO_BASE == "3D"):
                                
                                #Salva imagem representativa da calibração
                                cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_a_calibracao.png", imagem_original_reduzida)

                            #Salva imagem da área do pedúnculo RGB
                            cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_b_area_pedunculo.jpg", corteAreaPedunculo)
             
                cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_j_ponto_final.jpg", imagem_detectada)
                arquivo.write("\n")
                
            print("-*-*-*-*-*-*-*-*-*-*-*")
                        
            id_imagem = id_imagem + 1
                  
        except:

            print(f"{id_imagem} - Erro desconhecido\n") 
            
            traceback.print_exc()

            id_imagem = id_imagem + 1
            
    else:
        
        print("\n{} - Fruta nao identificada\n".format(id_imagem))
                
        naoIdentificadas += 1
        
        if(TIPO_BASE == "2D"):
    
            arquivo.write(f"{str(id_imagem)},-*-,-*-,-*-,-*-,-*-,-*-\n")
    
        else:
    
            arquivo.write(f"{str(id_imagem)},-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-,-*-\n")
    
        
        id_imagem = id_imagem + 1
        
tempoFinalGeral = time.time()
arquivo.write(f"\nTOTAL DE IMAGEM: {QUANTIDADE_IMAGEM}\n")
arquivo.write(f"Identificadas: {identificadas}\n")
arquivo.write(f"Não Identificadas: {naoIdentificadas}\n")
arquivo.write(f"Posição Ponto: {POSICAO_PONTO}\n")

arquivo.close()
print("\n\nFinalizando...")

print("Resultados: ")
print(f"Quantidade imagem: {quantidadeManga}")
print(f"Quantidade ponto correto: {pontoCorreto}")
print(f"Porcentagem acerto: {(pontoCorreto/quantidadeManga)*100}")
print(f"Quantidade ponto errado: {pontoErrado}")
print(f"Porcentagem errado: {(pontoErrado/quantidadeManga)*100}")

Executando...
Log...VALID_CUIT_POINT_3D/RESULTADO_RESULTADOS__MANGO_LEAFLESS
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/1_MANGO_LEAFLESS.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/1_MANGO_MAP_JSON.json

Imagem: 1 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.11467
Altura caixa: 130
Distancia Manga: 19.19 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 19.69 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 11
Quantidade de Pontos Candidatos: 221
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.55 cm
A distância do ponto selecionado para caixa conhecida: 2.98 cm
Porcentagem Mang

Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/8_MANGO_MAP_JSON.json

Imagem: 8 - Quantidade de fruta identificada: 2

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.21781
Altura caixa: 68
Distancia Manga: 36.23 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 75.49 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 11
Quantidade de Pontos Candidatos: 174
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.27 cm
A distância do ponto selecionado para caixa conhecida: 3.49 cm
Porcentagem Manga:     97.86 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.06 cm
Distancia Vertical:    0.38 cm
Porcentagem Ponto:     1 %
Manga localizada: 1
Função: --> Obter CM/PX
Fator cm_px: 0.19906
Altura caixa: 75
Distancia Manga: 31.23 cm
Fu

Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/15_MANGO_MAP_JSON.json

Imagem: 15 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.17914
Altura caixa: 83
Distancia Manga: 29.88 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 34.57 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 29
Quantidade de Pontos Candidatos: 247
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.41 cm
A distância do ponto selecionado para caixa conhecida: 3.95 cm
Porcentagem Manga:     91.05 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.55 cm
Distancia Vertical:    0.98 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho 

Imagem: 23 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.21723
Altura caixa: 69
Distancia Manga: 37.30 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 92.38 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 13
Quantidade de Pontos Candidatos: 151
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.7 cm
A distância do ponto selecionado para caixa conhecida: 4.15 cm
Porcentagem Manga:     90.61 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.09 cm
Distancia Vertical:    1.13 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/24_MANGO_LEAFLESS.jpg
Caminho do 

Quantidade de Pontos Candidatos: 310
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.24 cm
A distância do ponto selecionado para caixa conhecida: 2.67 cm
Porcentagem Manga:     99.48 %
Porcentagem Pedunculo: 100.0 %
Distancia Horizontal:  0.14 cm
Distancia Vertical:    0.35 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/32_MANGO_LEAFLESS.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/32_MANGO_MAP_JSON.json

Imagem: 32 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.10031
Altura caixa: 149
Distancia Manga: 16.75 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 468.75 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Fun

...

Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/123_MANGO_MAP_JSON.json

Imagem: 123 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.22160
Altura caixa: 67
Distancia Manga: 36.40 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 48.14 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 29
Quantidade de Pontos Candidatos: 66
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.33 cm
A distância do ponto selecionado para caixa conhecida: 3.1 cm
Porcentagem Manga:     93.5 %
Porcentagem Pedunculo: 46.43 %
Distancia Horizontal:  0.35 cm
Distancia Vertical:    0.12 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho d

Quantidade de Pontos Candidatos: 358
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.23 cm
A distância do ponto selecionado para caixa conhecida: 3.4 cm
Porcentagem Manga:     93.9 %
Porcentagem Pedunculo: 56.94 %
Distancia Horizontal:  0.52 cm
Distancia Vertical:    0.14 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/132_MANGO_LEAFLESS.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/132_MANGO_MAP_JSON.json

Imagem: 132 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.16227
Altura caixa: 92
Distancia Manga: 27.32 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 76.71 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Funç

Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/139_MANGO_MAP_JSON.json

Imagem: 139 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.13510
Altura caixa: 111
Distancia Manga: 22.47 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 29.37 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 29
Quantidade de Pontos Candidatos: 493
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.38 cm
A distância do ponto selecionado para caixa conhecida: 3.52 cm
Porcentagem Manga:     90.25 %
Porcentagem Pedunculo: 68.89 %
Distancia Horizontal:  0.2 cm
Distancia Vertical:    0.47 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminh

Quantidade de Pontos Candidatos: 36
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 0
Ponto no centro do area do pedúnculo
Posição definida: top
A distância do ponto selecionado para caixa IA: 3 cm
A distância do ponto selecionado para caixa conhecida: 2.9 cm
Porcentagem Manga:     94.23 %
Porcentagem Pedunculo: 62.0 %
Distancia Horizontal:  0.32 cm
Distancia Vertical:    0.25 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminho da imagem: VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/146_MANGO_LEAFLESS.jpg
Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/146_MANGO_MAP_JSON.json

Imagem: 146 - Quantidade de fruta identificada: 2

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.18110
Altura caixa: 82
Distancia Manga: 32.40 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 57.71 cm
Função: --> Segmenta Área Pedúnculo


Caminho do json:   VALID_CUIT_POINT_3D/VI3D_MANGO_LEAFLESS_SEM_FOLHA/152_MANGO_MAP_JSON.json

Imagem: 152 - Quantidade de fruta identificada: 1

Manga localizada: 0
Função: --> Obter CM/PX
Fator cm_px: 0.10678
Altura caixa: 140
Distancia Manga: 16.80 cm
Função: --> Prever area do pedunculo
Distancia Topo Caixa: 38.84 cm
Função: --> Segmenta Área Pedúnculo
Função --> Função Heuristica
Função: --> Encontra Pontos Candidatos
Função: --> Histograma HSV
Valor maximo HUE = 14
Quantidade de Pontos Candidatos: 74
Função: --> Obter CM/PX
Função: --> Seleciona pontos 3D
Quantidade de pontos para corte: 1
Posição definida: top
A distância do ponto selecionado para caixa IA: 3.14 cm
A distância do ponto selecionado para caixa conhecida: 2.96 cm
Porcentagem Manga:     92.91 %
Porcentagem Pedunculo: 49.46 %
Distancia Horizontal:  0.03 cm
Distancia Vertical:    0.09 cm
Porcentagem Ponto:     1 %
-*-*-*-*-*-*-*-*-*-*-*
############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################

Caminh